In [15]:
# LLM 호출
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template("what is the distance between {country_a} and {country_b}")

prompt = template.format(country_a= "Mexico", country_b="Thailand")

chat = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.1)

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers) when measured in a straight line.'

In [16]:
# LangChain 라이브러리에서 메시지 형식을 가져옵니다.
from langchain.schema import HumanMessage, AIMessage, SystemMessage  

# 메시지 리스트 생성
messages = [
    # SystemMessage: 시스템 메시지는 AI의 역할이나 설정을 정의합니다.
    # 여기서는 AI가 "지리 전문가"이며, 이탈리아어로만 대답해야 한다는 설정입니다.
    SystemMessage(content="You are a geography expert. AND you only reply in Italian"),
    
    # AIMessage: AI가 이전에 응답한 내용입니다. 
    # AI가 이미 "Ciao, mi chiamo Paolo!"라고 응답한 것으로 설정됩니다.
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    
    # HumanMessage: 사용자가 AI에게 질문한 내용입니다.
    # 여기서 사용자는 멕시코와 태국 사이의 거리와 AI의 이름을 묻습니다.
    HumanMessage(content="What is the distance between Mexico and Thailand. Also, what is your name")
]

# chat 객체에서 predict_messages 함수를 호출해 메시지를 예측
# messages 리스트를 입력으로 넣으면 AI가 설정에 맞게 다음 메시지를 예측합니다.
chat.predict_messages(messages)


AIMessage(content='Ciao! Il nome è Paolo. La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri.')

## Prompt


In [17]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in{language}."),
    ("ai", "Ciao, mi chiamo {name}!"),  # 튜플 형식
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?",),
])

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand"
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')

Output Parser
- LLM의 응답을 변형해야 할 때 필요하다

In [18]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):

    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

p = CommaOutputParser()

p.parse("Hello, how, are, you")



['Hello', 'how', 'are', 'you']

In [19]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else."),
    ("human", "{question}"),
])

# prompt = template.format_messages(max_items=10, question="What are the colors?")

# result = chat.predict_messages(prompt)

# p = CommaOutputParser()

# p.parse(result.content)

In [22]:
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 5,
    "question": "how many program language"
})



['Python', 'Java', 'C++', 'JavaScript', 'Ruby']